# Pre-processing

For pre processing, I have decided to do resizing of the images as it was seen during EDA that the images varied in size.


---


As we saw during the histogram plots, there was some disparity in the number of images among the classes. To address this issue, we may have to get the classes having less data at par with the others. Data Augmentation could be used here due to its capabilities to generate data from current data by adjusting some features of the current images.

I have added some basic augmentation, although I have decided to keep it optional (as it can be set to 'False') as we may want to train the model on the data as is, to compare performance.


---


Since the target values are of type string, we need to perform one-hot encoding on them. Hence, 'Class' and 'MoreThanOnePerson' have been encoded.

In [ ]:
df_reset = dfo.reset_index(drop=True)


In [ ]:
df_reset.head()

,FileName,Class,MoreThanOnePerson,HighLevelCategory
0,Images/Img_6056.jpg,walking_the_dog,YES,OutdoorActivities
1,Images/Img_8965.jpg,playing_guitar,NO,Artistic_MusicalActivities
2,Images/Img_6632.jpg,riding_a_bike,NO,Sports_Recreation
3,Images/Img_133.jpg,cutting_trees,NO,MaintenanceRepair
4,Images/Img_7178.jpg,feeding_a_horse,YES,AnimalCare


In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
from scipy.ndimage import rotate, shift
from sklearn.model_selection import train_test_split

def preprocess_image(image, augment=False):
    'method to preocess a single image: Resize and optionally apply augmentation'
    # Resize the image
    image = image.resize((224, 224))
    image = np.asarray(image)

    # Apply augmentations only if augment flag is True (for training data)
    if augment:
        # Random rotation between -10 and 10 degrees
        if np.random.rand() < 0.5:
            angle = np.random.randint(-10, 10)
            # Use scipy's rotate function to apply the rotation
            image = rotate(image, angle=angle, reshape=False)

            # shear
            shear_factor = np.random.uniform(-0.2, 0.2)
            image = tf.keras.preprocessing.image.apply_affine_transform(image, shear=shear_factor)

        # Random shift by -5 to 5 pixels
        shift_val = np.random.randint(-5, 5, size=2, dtype=int).tolist() + [0,]  # Shift in x, y only
        image = shift(image, shift_val, order=0, mode='constant', cval=0.0, prefilter=False)

    return image


def load_and_preprocess_data(data_frame, dim=(224, 224, 3), n_classes=40, data_mean=0, data_std=1, data_prefix='', augment=False):
    'Load and preprocess all data'
    num_samples = len(data_frame)
    X = np.empty((num_samples, *dim))
    y_activity = np.empty((num_samples, n_classes), dtype=int)
    y_person = np.empty((num_samples), dtype=int)

    # print(f"Shape of X: {X.shape}")
    # print(f"Shape of y_activity: {y_activity.shape}")
    # print(f"Shape of y_person: {y_person.shape}")

    # Create a mapping from category names to integers
    category_mapping = {category: idx for idx, category in enumerate(sorted(data_frame['Class'].unique()))}
    # Create a mapping from 'NO' and 'YES' to 0 and 1
    person_mapping = {'NO': 0, 'YES': 1}

    i=0
    for i, row in data_frame.iterrows():

        # print(f"Processing index: {i}")
        filepath = data_prefix + row['FileName']
        data = Image.open(filepath)

        # Preprocess images (resize and optionally augment)
        X[i] = preprocess_image(data, augment)
        y_activity[i] = tf.keras.utils.to_categorical(category_mapping[row['Class']], num_classes=n_classes)
        y_person[i] = person_mapping[row['MoreThanOnePerson']]

    return X, y_activity, y_person, category_mapping, person_mapping

# Now load and preprocess data correctly
X, y_activity, y_person, category_mapping, person_mapping = load_and_preprocess_data(
    df_reset, dim=(224, 224, 3), n_classes=40, data_mean=0, data_std=1, data_prefix='', augment=False
)



In [ ]:
# Confirm if the pre-processing went as required
print(X.shape)
print(y_activity.shape)
print(y_person.shape)

(3600, 224, 224, 3)
(3600, 40)
(3600,)


## Split dataset for hold-out validation

After conducting some Pre-processing, we shall now further split the dataset into training and validation before

After conducting some Pre-processing, we shall now further split the dataset into training and validation before

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_activity_train, y_activity_val, y_person_train, y_person_val = train_test_split(
    X, y_activity, y_person, test_size=0.2, random_state=42
)

Generate the train and validation datasets

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, {"activity_output": y_activity_train, "person_output": y_person_train}))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, {"activity_output": y_activity_val, "person_output": y_person_val}))

# Shuffle and batch the training dataset
batch_size = 16
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Batch the validation dataset
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)